In [9]:
#intraday check at the end of the day

import yfinance as yf
import pandas as pd
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

#fetching stock list from excel
excel_data_df = pd.read_excel('NSE\Monitored_Tickers.xlsx', sheet_name='Sheet1')
indian_tickers = excel_data_df['Ticker'].tolist()

# Function to check if the stock meets the criteria
def check_stock(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="max", interval="1d")
    #hist = stock.history(period="1y", interval="1d")
    
    if hist.empty or len(hist) < 2:
        return None
    
    current_price = hist['Close'][-1]
    all_time_high_till_yesterday = hist['High'][:-1].max()  # All-time high till yesterday
    todays_high = hist['High'][-1]
    todays_open = hist['Open'][-1]
    #breakout = FTweekhigh if all_time_high_till_yesterday > FTweekhigh else all_time_high_till_yesterday    
    
    if 50 < current_price < 3000:
        if current_price >= all_time_high_till_yesterday * 0.80:
        #if todays_high >= all_time_high_till_yesterday:# * 0.98:
            #volume = hist['Volume'][-1]
            #cmp_ath = current_price - all_time_high_till_yesterday
            #todays_high_cmp = todays_high - current_price

            return {
                'Ticker': ticker,
            }
    
    return None

# List to store the stocks that meet the criteria
filtered_stocks = []

# Loop through the tickers and apply the filter
for ticker in indian_tickers:
    result = check_stock(ticker)
    if result:
        filtered_stocks.append(result)

# Create a DataFrame from the results
filtered_stocks_df = pd.DataFrame(filtered_stocks)

# Sort the DataFrame by Volume in descending order and get the top 25 stocks
#filtered_stocks_df = filtered_stocks_df.sort_values(by='Volume', ascending=False)#.head(25)

# Calculate the sum of the 'CMP-ATH' column
#sum_cmp_ath = filtered_stocks_df['Profit'].sum()

# Print the DataFrame and the sum of 'CMP-ATH'
if not filtered_stocks_df.empty:
    print(filtered_stocks_df)
    #print(f"\nSum of 'CMP-ATH' column: {sum_cmp_ath}")
    filtered_stocks_df.to_csv('Short_List_Twenty.csv', index=False)
else:
    print("No stocks meet the criteria.")

$ABNINT.NS: possibly delisted; No timezone found
$AKANKSHA.NS: possibly delisted; No timezone found
$ARHAM.NS: possibly delisted; No timezone found
$BDR.NS: possibly delisted; No price data found  (1d 1925-08-22 -> 2024-07-29)


$BDR.NS: possibly delisted; No price data found  (1d 1925-08-22 -> 2024-07-29)


$BHALCHANDR.NS: possibly delisted; No timezone found
$EASTSUGIND.NS: possibly delisted; No timezone found
$FOCE.NS: possibly delisted; No timezone found
$GAMMONIND.NS: possibly delisted; No timezone found
$HINDZINC.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HIRECT.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HISARMETAL.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HITECH.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HITECHCORP.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HITECHGEAR.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HLEGLAS.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HLVLTD.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HMAAGRO.NS: possibly delisted; No price data found  (1d 1925-08-23 -> 2024-07-29)
$HMT.NS: possibly delisted; No price data

            Ticker
0     20MICRONS.NS
1    21STCENMGM.NS
2        360ONE.NS
3         AARON.NS
4       AARTECH.NS
..             ...
761      ZODIAC.NS
762      ZOMATO.NS
763        ZOTA.NS
764   ZYDUSLIFE.NS
765   ZYDUSWELL.NS

[766 rows x 1 columns]


In [19]:
import yfinance as yf
import pandas as pd
import warnings
import datetime
from tqdm import tqdm

# Ignore warnings
warnings.filterwarnings('ignore')

# Load stock list from Excel file
excel_file_path = 'Short_List_Twenty.xlsx'
sheet_name = 'Short_List_Twenty'

# Read the Excel file into a DataFrame
stock_list_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Extract the 'Ticker' column and convert it to a list
indian_stock_list = stock_list_df['Ticker'].tolist()

# Function to check stock details and identify profitable stocks
def check_stock(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="max", interval="1d")
    print(hist)
    if hist.empty or len(hist) < 2:
        return None
    keys_to_extract = [
        'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'maxAge', 'priceHint',
        'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 
        'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'payoutRatio', 'beta', 'trailingPE', 
        'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 
        'bid', 'ask', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 
        'fiftyDayAverage', 'twoHundredDayAverage', 'enterpriseValue', 'profitMargins', 'floatShares', 
        'sharesOutstanding', 'heldPercentInsiders', 'heldPercentInstitutions', 'impliedSharesOutstanding', 
        'bookValue', 'priceToBook', 'lastFiscalYearEnd', 'nextFiscalYearEnd', 'mostRecentQuarter', 
        'earningsQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'lastSplitFactor', 
        'lastSplitDate', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange', 'SandP52WeekChange', 
        'lastDividendValue', 'lastDividendDate', 'shortName', 'longName', 'firstTradeDateEpochUtc', 
        'gmtOffSetMilliseconds', 'currentPrice', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 
        'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'numberOfAnalystOpinions', 'totalCash', 
        'totalCashPerShare', 'ebitda', 'totalDebt', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 
        'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins'
    ]
    All_data = {}
    try:
        fifty_two_week_high = stock.info['fiftyTwoWeekHigh']
    except KeyError:
        fifty_two_week_high = 0

    

    current_price = hist['Close'][-1]
    all_time_high_till_yesterday = hist['High'][:-1].max()
    todays_high = hist['High'][-1]
    todays_open = hist['Open'][-1]

    breakout_level = fifty_two_week_high if all_time_high_till_yesterday > fifty_two_week_high and fifty_two_week_high != 0 else all_time_high_till_yesterday

    if 50 < current_price < 3000 and todays_high >= breakout_level + 1:
        volume = hist['Volume'][-1]

        if todays_open > breakout_level:
            entry_price = todays_open
            stop_loss = breakout_level - 1
        else:
            entry_price = breakout_level
            stop_loss = max(todays_open, breakout_level * 0.99) - 1

        if entry_price > 100:
            Trailing_Stop_Loss = todays_high - 5
        else:
            Trailing_Stop_Loss = entry_price - (todays_high * 0.05)

        Trailing_Stop_Loss= max(Trailing_Stop_Loss,stop_loss)
        exit_price = max(current_price, stop_loss)
        stop_loss_hit = "Yes" if current_price < stop_loss else "No"
        profit = exit_price - entry_price
        moving_avg_50 = hist['Close'].rolling(window=50).mean().iloc[-1]
        moving_avg_200 = hist['Close'].rolling(window=200).mean().iloc[-1]
        extracted_values = {key: stock.info.get(key) for key in keys_to_extract}
        All_data.update(extracted_values)

        data = {
            'Ticker': ticker,
            'All Time High': all_time_high_till_yesterday,
            '52 Week High': fifty_two_week_high,
            'Breakout Level': breakout_level,
            'Today\'s High': todays_high,
            'Today\'s Open': todays_open,
            'Current Market Price': current_price,
            'Stop Loss': stop_loss,
            'Stop Loss Points': entry_price - stop_loss,
            'Stop Loss Hit': stop_loss_hit,
            'Entry Price': entry_price,
            'Exit Price': exit_price,
            'Profit': profit,
            'Trailing Exit': Trailing_Stop_Loss,
            'Trailing Profit': Trailing_Stop_Loss - entry_price,
            '50 Day MA': moving_avg_50,
            '200 Day MA': moving_avg_200
        }
        All_data.update(data)

    return All_data

# List to store filtered stocks
filtered_stocks = []

# Loop through the tickers and apply the filter with a progress bar
for ticker in tqdm(indian_stock_list, desc="Processing stocks"):
    result = check_stock(ticker)
    if result:
        filtered_stocks.append(result)

# Create a DataFrame from the results
filtered_stocks_df = pd.DataFrame(filtered_stocks)

# Sort the DataFrame by Volume in descending order
filtered_stocks_df = filtered_stocks_df.sort_values(by='Profit', ascending=False)

# Calculate the total profit and other metrics
total_profit = filtered_stocks_df['Profit'].sum()
total_trailing_sl = filtered_stocks_df['Trailing Exit'].sum()
total_entry_capital = filtered_stocks_df['Entry Price'].sum()
total_return_percentage = (total_profit / total_entry_capital) * 100
leverage_capital = total_entry_capital * 0.2
total_return_with_leverage = (total_profit / leverage_capital) * 100

# Print the DataFrame and calculated metrics
if not filtered_stocks_df.empty:
    print(filtered_stocks_df)
    print(f"\nToday's Profit: {total_profit}")
    print(f"\nTrailing Stop Loss High Profit: {total_trailing_sl}")
    print(f"\nTotal Capital Used: {total_entry_capital}")
    print(f"\nTotal Return for the Day: {total_return_percentage:.2f}%")
    print(f"\nWith 80% Leverage, Capital Used: {leverage_capital}")
    print(f"\nTotal Return for the Day with Leverage: {total_return_with_leverage:.2f}%")

    timestamp = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M")
    filtered_stocks_df.to_csv(f'strategy_results_{timestamp}.csv', index=False)
else:
    print("No stocks meet the criteria.")

Processing stocks:   0%|          | 0/836 [00:00<?, ?it/s]

                                 Open        High         Low       Close  \
Date                                                                        
2008-10-06 00:00:00+05:30   34.939767   34.939767   13.801208   14.652864   
2008-10-07 00:00:00+05:30   13.975905   16.596387   12.163405   13.124248   
2008-10-08 00:00:00+05:30   12.228918   12.753015   10.962352   11.573797   
2008-10-10 00:00:00+05:30   10.875002   10.875002    9.455574   10.132532   
2008-10-13 00:00:00+05:30   10.612953   11.617472   10.176206   10.765815   
...                               ...         ...         ...         ...   
2024-07-23 00:00:00+05:30  212.850006  231.139999  211.000000  218.770004   
2024-07-24 00:00:00+05:30  219.649994  229.500000  219.649994  222.669998   
2024-07-25 00:00:00+05:30  221.800003  224.710007  216.320007  218.399994   
2024-07-26 00:00:00+05:30  219.350006  262.079987  219.250000  257.570007   
2024-07-29 00:00:00+05:30  263.010010  294.799988  256.100006  277.899994   

Processing stocks:   0%|          | 1/836 [00:00<11:36,  1.20it/s]

                                Open       High        Low      Close  Volume  \
Date                                                                            
2018-01-11 00:00:00+05:30  30.162642  30.162642  30.162642  30.162642     157   
2018-01-12 00:00:00+05:30  30.749638  30.749638  30.749638  30.749638     360   
2018-01-15 00:00:00+05:30  31.336634  31.336634  31.336634  31.336634     795   
2018-01-16 00:00:00+05:30  31.923630  31.923630  31.923630  31.923630    2735   
2018-01-17 00:00:00+05:30  32.555782  32.555782  31.968787  32.555782   56033   
...                              ...        ...        ...        ...     ...   
2024-07-23 00:00:00+05:30  73.686951  73.686951  73.686951  73.686951   20727   
2024-07-24 00:00:00+05:30  75.158951  75.158951  75.158951  75.158951    2505   
2024-07-25 00:00:00+05:30  76.660004  76.660004  76.660004  76.660004     508   
2024-07-26 00:00:00+05:30  78.190002  78.190002  78.190002  78.190002     595   
2024-07-29 00:00:00+05:30  7

Processing stocks:   0%|          | 2/836 [00:01<07:56,  1.75it/s]

...

Processing stocks:   1%|▏         | 11/836 [00:05<07:19,  1.88it/s]

                                 Open        High         Low       Close  \
Date                                                                        
2022-11-21 00:00:00+05:30  443.607805  469.238478  433.947024  451.642029   
2022-11-22 00:00:00+05:30  454.254379  471.210067  443.607804  448.388916   
2022-11-23 00:00:00+05:30  452.381369  464.950257  449.128258  458.098999   
2022-11-24 00:00:00+05:30  461.352141  501.769743  458.394755  495.263489   
2022-11-25 00:00:00+05:30  488.905062  531.146355  486.194125  526.414551   
...                               ...         ...         ...         ...   
2024-07-23 00:00:00+05:30  712.700012  712.700012  660.049988  699.599976   
2024-07-24 00:00:00+05:30  691.500000  747.400024  691.500000  741.549988   
2024-07-25 00:00:00+05:30  728.849976  738.000000  718.750000  724.599976   
2024-07-26 00:00:00+05:30  726.750000  737.000000  720.099976  732.750000   
2024-07-29 00:00:00+05:30  735.000000  787.000000  732.750000  773.799988   

Processing stocks:   1%|▏         | 12/836 [00:06<07:27,  1.84it/s]


                                  Open         High          Low        Close  \
Date                                                                            
2023-08-16 00:00:00+05:30   795.000000   822.900024   795.000000   818.099976   
2023-08-17 00:00:00+05:30   819.849976   836.450012   819.000000   822.200012   
2023-08-18 00:00:00+05:30   824.900024   894.000000   824.750000   871.900024   
2023-08-21 00:00:00+05:30   879.000000   932.000000   878.000000   923.549988   
2023-08-22 00:00:00+05:30   945.099976   976.950012   923.849976   960.150024   
...                                ...          ...          ...          ...   
2024-07-23 00:00:00+05:30  1010.099976  1033.199951   975.000000  1023.799988   
2024-07-24 00:00:00+05:30  1025.900024  1047.000000  1022.000000  1036.199951   
2024-07-25 00:00:00+05:30  1030.199951  1060.050049  1025.050049  1050.150024   
2024-07-26 00:00:00+05:30  1063.050049  1149.699951  1040.050049  1051.949951   
2024-07-29 00:00:00+05:30  1

KeyboardInterrupt: 

In [29]:
import yfinance as yf
import pandas as pd

# Define the list of stock tickers (example with a few tickers, add up to 1000)
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA']  # Add more tickers up to 1000

# Define the keys to extract
keys_to_extract = [
    'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'maxAge', 'priceHint',
    'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 
    'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'payoutRatio', 'beta', 'trailingPE', 
    'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 
    'bid', 'ask', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 
    'fiftyDayAverage', 'twoHundredDayAverage', 'enterpriseValue', 'profitMargins', 'floatShares', 
    'sharesOutstanding', 'heldPercentInsiders', 'heldPercentInstitutions', 'impliedSharesOutstanding', 
    'bookValue', 'priceToBook', 'lastFiscalYearEnd', 'nextFiscalYearEnd', 'mostRecentQuarter', 
    'earningsQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'lastSplitFactor', 
    'lastSplitDate', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange', 'SandP52WeekChange', 
    'lastDividendValue', 'lastDividendDate', 'shortName', 'longName', 'firstTradeDateEpochUtc', 
    'gmtOffSetMilliseconds', 'currentPrice', 'targetHighPrice', 'targetLowPrice', 'targetMeanPrice', 
    'targetMedianPrice', 'recommendationMean', 'recommendationKey', 'numberOfAnalystOpinions', 'totalCash', 
    'totalCashPerShare', 'ebitda', 'totalDebt', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 
    'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins'
]

# Initialize an empty DataFrame to store data
all_stock_data = pd.DataFrame()

# Fetch data for each ticker
for ticker in tickers:
    try:
        # Get the ticker data
        ticker_data = yf.Ticker(ticker).info
        
        # Extract the relevant keys
        extracted_data = {key: ticker_data.get(key, None) for key in keys_to_extract}
        extracted_data['Ticker'] = ticker
        
        # Append to the DataFrame
        all_stock_data = all_stock_data.append(extracted_data, ignore_index=True)
        print (all_stock_data)
    except Exception as e:
        print(f"Could not retrieve data for {ticker}: {e}")

# Save to an Excel file
output_file = 'bulk_stock_data.xlsx'
all_stock_data.to_excel(output_file, index=False)

print(f"Data saved to {output_file}")


Could not retrieve data for AAPL: 'DataFrame' object has no attribute 'append'
Could not retrieve data for MSFT: 'DataFrame' object has no attribute 'append'
Could not retrieve data for GOOG: 'DataFrame' object has no attribute 'append'
Could not retrieve data for AMZN: 'DataFrame' object has no attribute 'append'
Could not retrieve data for TSLA: 'DataFrame' object has no attribute 'append'


PermissionError: [Errno 13] Permission denied: 'bulk_stock_data.xlsx'

In [21]:
import yfinance as yf
import pandas as pd

# Define the list of stock tickers (example with 1000 tickers)
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA']  # Add more tickers up to 1000

# Fetch data for all tickers in a single call
stock_data = yf.download(tickers, period='max')

# If you want to have a flat DataFrame with a ticker column
flat_stock_data = stock_data.stack(level=1).reset_index()

# Save to an Excel file
output_file = 'bulk_stock_data.xlsx'
flat_stock_data.to_excel(output_file, index=False)

print(f"Data saved to {output_file}")


[*********************100%%**********************]  5 of 5 completed


Data saved to bulk_stock_data.xlsx


In [23]:
import yfinance as yf
import pandas as pd

# Define the list of stock tickers (example with a few tickers, add up to 1000)
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA']  # Add more tickers up to 1000

# Fetch historical data for all tickers in a single call
stock_data = yf.download(tickers, period='max')

# Initialize lists to store the all-time high and low data
all_time_highs = []
all_time_lows = []

# Compute all-time high and low for each ticker
for ticker in tickers:
    ticker_data = stock_data['Adj Close'][ticker]
    all_time_high = ticker_data.max()
    all_time_low = ticker_data.min()
    all_time_highs.append({'Ticker': ticker, 'AllTimeHigh': all_time_high})
    all_time_lows.append({'Ticker': ticker, 'AllTimeLow': all_time_low})

# Convert the lists to DataFrames
all_time_highs_df = pd.DataFrame(all_time_highs)
all_time_lows_df = pd.DataFrame(all_time_lows)

# Merge the high and low DataFrames
all_time_data = pd.merge(all_time_highs_df, all_time_lows_df, on='Ticker')

# Save to an Excel file
output_file = 'all_time_highs_lows.xlsx'
all_time_data.to_excel(output_file, index=False)

print(f"All-time highs and lows data saved to {output_file}")


[*********************100%%**********************]  5 of 5 completed

All-time highs and lows data saved to all_time_highs_lows.xlsx


In [18]:
print(hist)

NameError: name 'hist' is not defined

In [15]:
# Load the Excel file
file_path = 'data_analysis.xlsx'
data = pd.read_excel(file_path, sheet_name=None)

# Combine all sheets into one DataFrame
combined_data = pd.concat(data.values(), ignore_index=True)
# Filter for stocks with higher than average volume
high_volume_stocks = combined_data[combined_data['volume'] > combined_data['averageVolume']]

# Find stocks with a Golden Cross
golden_cross_stocks = combined_data[combined_data['fiftyDayAverage'] > combined_data['twoHundredDayAverage']]

# Filter for stocks with favorable valuation ratios and strong growth metrics
filtered_stocks = combined_data[(combined_data['trailingPE'] < 15) & (combined_data['earningsQuarterlyGrowth'] > 0.2)]

# Select stocks with positive analyst sentiment
analyst_favorites = combined_data[(combined_data['recommendationMean'] < 3) & (combined_data['targetMeanPrice'] > combined_data['currentPrice'])]

# Combine all filters
final_selection = high_volume_stocks.merge(golden_cross_stocks, how='inner').merge(filtered_stocks, how='inner').merge(analyst_favorites, how='inner')

# Display the final selected stocks
print(final_selection)

     auditRisk  boardRisk  compensationRisk  shareHolderRightsRisk  \
0          NaN        NaN               NaN                    NaN   
1          NaN        NaN               NaN                    NaN   
3          NaN        NaN               NaN                    NaN   
4          NaN        NaN               NaN                    NaN   
5          NaN        NaN               NaN                    NaN   
..         ...        ...               ...                    ...   
336        NaN        NaN               NaN                    NaN   
337        NaN        NaN               NaN                    NaN   
338        NaN        NaN               NaN                    NaN   
339        NaN        NaN               NaN                    NaN   
340        NaN        NaN               NaN                    NaN   

     overallRisk  maxAge  priceHint  previousClose     open   dayLow  ...  \
0            NaN   86400        2.0        1864.00  1891.05  1891.00  ...   
1    

In [16]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm

# Load data from Excel
excel_file_path = 'data_analysis.xlsx'
sheets = pd.ExcelFile(excel_file_path).sheet_names

# Function to check stock details and identify profitable stocks
def check_stock(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1y", interval="1d")
    if hist.empty or len(hist) < 2:
        return None
    
    current_price = hist['Close'][-1]
    fifty_two_week_high = stock.info.get('fiftyTwoWeekHigh', 0)
    average_volume = stock.info.get('averageVolume', 0)
    current_volume = hist['Volume'][-1]
    moving_avg_50 = hist['Close'].rolling(window=50).mean().iloc[-1]
    moving_avg_200 = hist['Close'].rolling(window=200).mean().iloc[-1]
    profit_margin = stock.info.get('profitMargins', 0)
    revenue_growth = stock.info.get('revenueGrowth', 0)
    earnings_growth = stock.info.get('earningsGrowth', 0)
    
    if (current_volume > average_volume and
        current_price >= 0.95 * fifty_two_week_high and
        current_price > moving_avg_50 > moving_avg_200 and
        profit_margin > 0.1 and
        revenue_growth > 0 and
        earnings_growth > 0):
        
        return {
            'Ticker': ticker,
            'Current Price': current_price,
            '52 Week High': fifty_two_week_high,
            'Average Volume': average_volume,
            'Current Volume': current_volume,
            '50 Day MA': moving_avg_50,
            '200 Day MA': moving_avg_200,
            'Profit Margin': profit_margin,
            'Revenue Growth': revenue_growth,
            'Earnings Growth': earnings_growth
        }
    return None

# List to store filtered stocks
filtered_stocks = []

# Loop through sheets and apply the filter with a progress bar
for sheet in sheets:
    stock_list_df = pd.read_excel(excel_file_path, sheet_name=sheet)
    indian_stock_list = stock_list_df['Ticker'].tolist()
    
    for ticker in tqdm(indian_stock_list, desc=f"Processing stocks in {sheet}"):
        result = check_stock(ticker)
        if result:
            filtered_stocks.append(result)

# Create a DataFrame from the results
filtered_stocks_df = pd.DataFrame(filtered_stocks)

# Sort the DataFrame by Current Price in descending order
filtered_stocks_df = filtered_stocks_df.sort_values(by='Current Price', ascending=False)

# Save the DataFrame to a CSV file
timestamp = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M")
# filtered_stocks_df.to_csv(f'enhanced_strategy_results_{timestamp}.csv', index=False)

# Print the DataFrame
if not filtered_stocks_df.empty:
    print(filtered_stocks_df)
else:
    print("No stocks meet the criteria.")


Processing stocks in strategy_results_29_07_2024_11_:  88%|████████▊ | 78/89 [00:28<00:04,  2.65it/s]RKSWAMY.NS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
Processing stocks in strategy_results_29_07_2024_11_: 100%|██████████| 89/89 [00:32<00:00,  2.74it/s]
Processing stocks in strategy_results_29_07_2024_10_:  72%|███████▏  | 62/86 [00:19<00:07,  3.21it/s]RKSWAMY.NS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
Processing stocks in strategy_results_29_07_2024_10_: 100%|██████████| 86/86 [00:27<00:00,  3.16it/s]
Processing stocks in strategy_results_29_07_2024_09_:  16%|█▌        | 12/75 [00:02<00:14,  4.40it/s]RKSWAMY.NS: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
Processing stocks in strategy_results_29_07_2024_09_: 100%|██████████| 75/75 [00:21<00:00,  3.48it/s]
Processing stocks in strategy_results_26_07_2024_18_:  57%|█████▋    | 52/91 [00:16<00:13,  2.

           Ticker  Current Price  52 Week High  Average Volume  \
29        JASH.NS    2574.000000       2595.00           49687   
36        JASH.NS    2573.949951       2595.00           49687   
11        JASH.NS    2570.550049       2595.00           49687   
0         JASH.NS    2567.949951       2595.00           49687   
3   JBCHEPHARM.NS    1920.400024       1958.00          196765   
30      ZENTEC.NS    1609.050049       1609.05          553975   
10      ZENTEC.NS    1609.050049       1609.05          553975   
14      ZENTEC.NS    1609.050049       1609.05          553975   
21      ZENTEC.NS    1609.050049       1609.05          553975   
34       CIPLA.NS    1544.349976       1600.00         2089267   
1         ERIS.NS    1117.050049       1134.30          215108   
12        ERIS.NS    1116.949951       1134.30          215108   
26        ERIS.NS    1116.949951       1134.30          215108   
35        ERIS.NS    1116.599976       1134.30          215108   
16     EMU